In [1]:
import torch

from torch_geometric.utils import to_dense_adj
from torch.utils.data import DataLoader, random_split
from torch_geometric.nn.pool import global_mean_pool, global_max_pool

In [2]:
from helpers import CVFConfigDataset

In [3]:
dataset = CVFConfigDataset(
    "dijkstra",
    "implicit_graph_n10_config_rank_dataset.csv",
    "implicit_graph_n10_edge_index.json",
    3,
    one_hot_encode=False,
)

train_split = 0.9
dev_split = 0.05
test_split = 0.05


train_set, dev_set, test_set = random_split(
    dataset, [train_split, dev_split, test_split]
)

train_loader = DataLoader(train_set, batch_size=1024, shuffle=False)
dev_loader = DataLoader(dev_set, batch_size=1024, shuffle=False)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
adjacency = to_dense_adj(dataset.edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency = adjacency.unsqueeze(0).to(device)
adjacency

tensor([[[1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1., 1.]]], device='cuda:0')

In [6]:
def add_graph_properties(x):
    """ this needs to be optimized """
    inp = torch.tensor([]).to(device)
    for each in x:
        inp = torch.cat((inp, torch.mul(each, adjacency)))
    return inp

In [23]:
class MLPNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.layer1 = torch.nn.Linear(dim_in, dim_h)
        self.layer2 = torch.nn.Linear(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x):
        h = self.layer1(x)
        h = torch.relu(h)
        h = self.layer2(h)
        h = torch.relu(h)
        h = self.out(h)
        # h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        h = h.mean()
        h = torch.sigmoid(h)
        # h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, train_loader, validation_loader, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        # criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.01) # weight_decay is a L2 regularization parameter
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in train_loader:
                x = batch[0].to(device)
                x = add_graph_properties(x)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1)
                y = (y > 0).long()
                optimizer.zero_grad()
                out = self(x)
                print("output", out.shape, "y", y.shape)
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Training set | Epoch:", epoch, "Loss:", total_loss / count)
            
            # self.eval()
            # with torch.no_grad():
            #     total_loss = 0
            #     count = 0
            #     for batch in validation_loader:
            #         x = batch[0].to(device)
            #         x = add_graph_properties(x)
            #         y = batch[1].to(device)
            #         y = (y > 0).float()
            #         optimizer.zero_grad()
            #         out = self(x)
            #         loss = criterion(out, y)
            #         total_loss += loss
            #         count += 1

            # print("Validatn set | Epoch:", epoch, "Loss:", total_loss / count)
            # print()


In [24]:
mlp = MLPNN(dataset.nodes, 20, 1).to(device)
print(mlp)
print()

mlp.fit(train_loader, dev_loader, epochs=10)

MLPNN(
  (layer1): Linear(in_features=10, out_features=20, bias=True)
  (layer2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=1, bias=True)
)

output torch.Size([]) y torch.Size([1024, 1, 1])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [30]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

total_matched = 0
edge_index = dataset.edge_index.t().to(device)

for batch in test_loader:
    x = batch[0].to(device)
    x = add_graph_properties(x)
    y = batch[1].to(device)
    y = (y > 0).long()
    # y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = mlp(x)
    predicted = torch.round(predicted)
    print(predicted.shape, y.shape)
    matched = (predicted == y).sum().item()
    total_matched += matched

print("Total matched", total_matched, "out of", len(test_set), "| Accuracy", round(total_matched/len(test_set) * 100, 4), "%")

torch.Size([128, 10, 1]) torch.Size([128, 1])


RuntimeError: The size of tensor a (10) must match the size of tensor b (128) at non-singleton dimension 1